# Model Training with K-Folds Cross-Validation

In [1]:
%load_ext autoreload
%autoreload 2

import joblib
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
import pandas as pd

from pathlib import Path
from sklearn import ensemble
from typing import Dict, Tuple

from evidently.metrics import RegressionQualityMetric, RegressionErrorPlot, RegressionErrorDistribution
from evidently.metric_preset import DataDriftPreset, RegressionPreset
from evidently.pipeline.column_mapping import ColumnMapping
from evidently.report import Report

from config import MLFLOW_TRACKING_URI, DATA_DIR, FILENAME, REPORTS_DIR

/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnrozhkov/dev/mlrepa/com/evidently/evidently-mlflow/.venv/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/mnr

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [3]:
# Download original dataset with: python src/pipelines/load_data.py 

raw_data = pd.read_csv(f"../{DATA_DIR}/{FILENAME}")

raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Define column mapping

In [4]:
target = 'cnt'
prediction = 'prediction'
datetime = 'dteday'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

column_mapping = ColumnMapping()
column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.datetime = datetime
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

# Define the comparison windows

In [5]:
start_date_0 = '2011-01-02 00:00:00'
end_date_0 = '2011-01-30 23:00:00'

experiment_batches = [
    
    ('2011-01-31 00:00:00','2011-02-06 23:00:00'),
    ('2011-02-07 23:00:00','2011-02-13 23:00:00'),
    ('2011-02-14 23:00:00','2011-02-20 23:00:00'),
    ('2011-02-21 00:00:00','2011-02-27 23:00:00'),
    ('2011-02-28 00:00:00','2011-03-06 23:00:00'),  
]

## Define the Reference data

In [6]:
# Set datetime index 
raw_data = raw_data.set_index('dteday')

# Define the reference dataset
reference = raw_data.loc[start_date_0:end_date_0]

print(reference.shape)
reference.head()

(617, 16)


,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-03,48,1,0,1,0,0,1,1,1,0.22,0.1970,0.44,0.3582,0,5,5
2011-01-03,49,1,0,1,1,0,1,1,1,0.20,0.1667,0.44,0.4179,0,2,2
2011-01-03,50,1,0,1,4,0,1,1,1,0.16,0.1364,0.47,0.3881,0,1,1
2011-01-03,51,1,0,1,5,0,1,1,1,0.16,0.1364,0.47,0.2836,0,3,3
2011-01-03,52,1,0,1,6,0,1,1,1,0.14,0.1061,0.50,0.3881,0,30,30


## Train Models

In [7]:
# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment('Train Model - K-Fold')

# Set experiment variables
model_path = Path('../models/model.joblib')
ref_end_data = end_date_0

# Run model train for each batch (K-Fold)
for k, date in enumerate(experiment_batches):

    print(f"Train period: {start_date_0} - {ref_end_data}") 
    X_train = raw_data.loc[start_date_0:ref_end_data, numerical_features + categorical_features]
    y_train = raw_data.loc[start_date_0:ref_end_data, target]
    print("X_train (reference) dataset shape: ", X_train.shape, y_train.shape)
    
    print(f"Test period: {date[0]} - {date[1]}") 
    current = raw_data.loc[date[0]:date[1]]
    X_test = current.loc[:, numerical_features + categorical_features]
    y_test = current[target]
    print("X_test (current)) dataset shape: ",  X_test.shape, y_test.shape)
    
    # Update reference end date
    ref_end_data = date[1]
    
    # Train model
    regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
    regressor.fit(X_train, y_train)
    
    # Make predictions 
    ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
    reference['prediction'] = ref_prediction
    
    prediction = regressor.predict(current[numerical_features + categorical_features])
    current['prediction'] = prediction
    
    # Build the model performance report
    regression_performance_report = Report(metrics=[
        RegressionQualityMetric(),
    ])
    regression_performance_report.run(
        reference_data=reference, 
        current_data=current,
        column_mapping=column_mapping)
    
    # Extract Metrics from the report
    train_report_metrics = regression_performance_report.as_dict()
    me = train_report_metrics['metrics'][0]['result']['current']['mean_error']
    mae = train_report_metrics['metrics'][0]['result']['current']["mean_abs_error"]
    
    # Save model and train_report
    joblib.dump(regressor, model_path)
    model_quality_report_path = f"../{REPORTS_DIR}/train_report.html"
    regression_performance_report.save_html(model_quality_report_path)

    # Start a new MLflow Run
    with mlflow.start_run() as run: 
        
        # Show newly created run metadata info
        print("Experiment id: {}".format(run.info.experiment_id))
        print("Run id: {}".format(run.info.run_id))
        print("Run name: {}".format(run.info.run_name))
        print('MLFlow tracking uri:', mlflow.get_tracking_uri())
        print('MLFlow artifact uri:', mlflow.get_artifact_uri())
        
        # Log parameters
        mlflow.log_param("begin", date[0])
        mlflow.log_param("end", date[1])
        
        # Log metrics
        mlflow.log_metric('me', round(me, 3))
        mlflow.log_metric('mae', round(mae, 3))
        
        # Log model 
        mlflow.log_artifact(model_path)
        
        # Log the regression_performance_report as an artifact
        mlflow.log_artifact(model_quality_report_path)

2023/07/11 16:09:26 INFO mlflow.tracking.fluent: Experiment with name 'Train Model - K-Fold' does not exist. Creating a new experiment.


Client tracking uri: http://localhost:5000
Train period: 2011-01-02 00:00:00 - 2011-01-30 23:00:00
X_train (reference) dataset shape:  (617, 10) (617,)
Test period: 2011-01-31 00:00:00 - 2011-02-06 23:00:00
X_test (current)) dataset shape:  (141, 10) (141,)
Experiment id: 653138467686023976
Run id: 5964cb08fdbb4754875ea537a8520199
Run name: auspicious-mule-10
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/653138467686023976/5964cb08fdbb4754875ea537a8520199/artifacts
Train period: 2011-01-02 00:00:00 - 2011-02-06 23:00:00
X_train (reference) dataset shape:  (782, 10) (782,)
Test period: 2011-02-07 23:00:00 - 2011-02-13 23:00:00
X_test (current)) dataset shape:  (139, 10) (139,)
Experiment id: 653138467686023976
Run id: a6e4e9581bc7434b9056d5d16ddcd8b5
Run name: rebellious-foal-972
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/653138467686023976/a6e4e9581bc7434b9056d5d16ddcd8b5/artifacts
Train period: 2011-01-02 00:00

## Implementing K-Fold Cross-Validation Runs Using MLFlow's Nested Runs Feature

In [8]:
# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment('Train Model - K-Fold')

# Set experiment variables
model_path = Path('../models/model.joblib')
ref_end_data = end_date_0

# Start a new Run (Parent Run)
with mlflow.start_run() as run: 
    
    # Update metrics with metrics for each Fold
    metrics = {}

    # Run model train for each batch (K-Fold)
    for k, date in enumerate(experiment_batches):
        
        # print(k, date[0],  date[1])
            
        print(f"Train period: {start_date_0} - {ref_end_data}") 
        X_train = raw_data.loc[start_date_0:ref_end_data, numerical_features + categorical_features]
        y_train = raw_data.loc[start_date_0:ref_end_data, target]
        print("X_train (reference) dataset shape: ", X_train.shape, y_train.shape)
        
        print(f"Test period: {date[0]} - {date[1]}") 
        current = raw_data.loc[date[0]:date[1]]
        X_test = current.loc[:, numerical_features + categorical_features]
        y_test = current[target]
        print("X_test (current)) dataset shape: ",  X_test.shape, y_test.shape)
        
        # Update reference end date
        ref_end_data = date[1]

        # Train model
        regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
        regressor.fit(X_train, y_train)
        
        # Make predictions 
        ref_prediction = regressor.predict(reference[numerical_features + categorical_features])
        reference['prediction'] = ref_prediction
        
        prediction = regressor.predict(current[numerical_features + categorical_features])
        current['prediction'] = prediction 
        
        # Build the model performance report
        regression_performance_report = Report(metrics=[
            RegressionQualityMetric(),
        ])
        regression_performance_report.run(
            reference_data=reference, 
            current_data=current,
            column_mapping=column_mapping)
        
        # Extract Metrics from the report
        train_report_metrics = regression_performance_report.as_dict()
        me = train_report_metrics['metrics'][0]['result']['current']['mean_error']
        mae = train_report_metrics['metrics'][0]['result']['current']["mean_abs_error"]
        metrics.update({date[1]: {'me': me, 'mae': mae}})
        
        # Save train_report for (the Fold)
        model_quality_report_path = f"../{REPORTS_DIR}/train_report.html"
        regression_performance_report.save_html(model_quality_report_path)
        
        # Run a Child Run for each Fold 
        with mlflow.start_run(run_name=date[1], nested=True) as child_run:
            
            # Show newly created run metadata info
            print("Experiment id: {}".format(run.info.experiment_id))
            print("Run id: {}".format(run.info.run_id))
            print("Run name: {}".format(run.info.run_name))
            print('MLFlow tracking uri:', mlflow.get_tracking_uri())
            print('MLFlow artifact uri:', mlflow.get_artifact_uri())
            
            # Log parameters
            mlflow.log_param("begin", date[0])
            mlflow.log_param("end", date[1])
            
            # Log metrics
            mlflow.log_metric('me', round(me, 3))
            mlflow.log_metric('mae', round(mae, 3))
            
            # Log the regression_performance_report as an artifact
            mlflow.log_artifact(model_quality_report_path)
        
    # Save model
    joblib.dump(regressor, model_path)
    
    # Log the last batch model as the parent Run model
    mlflow.log_artifact(model_path)
    
    # Log metrics
    average_run_merics = pd.DataFrame.from_dict(metrics).T.mean().round(3).to_dict()
    mlflow.log_metrics(average_run_merics )

Client tracking uri: http://localhost:5000
Train period: 2011-01-02 00:00:00 - 2011-01-30 23:00:00
X_train (reference) dataset shape:  (617, 10) (617,)
Test period: 2011-01-31 00:00:00 - 2011-02-06 23:00:00
X_test (current)) dataset shape:  (141, 10) (141,)
Experiment id: 653138467686023976
Run id: a5ef43800e3549aeb4b1549202900bbf
Run name: clean-bird-126
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/653138467686023976/a7f75010b9d64a4da7ef92ae611c4b1c/artifacts
Train period: 2011-01-02 00:00:00 - 2011-02-06 23:00:00
X_train (reference) dataset shape:  (782, 10) (782,)
Test period: 2011-02-07 23:00:00 - 2011-02-13 23:00:00
X_test (current)) dataset shape:  (139, 10) (139,)
Experiment id: 653138467686023976
Run id: a5ef43800e3549aeb4b1549202900bbf
Run name: clean-bird-126
MLFlow tracking uri: http://localhost:5000
MLFlow artifact uri: mlflow-artifacts:/653138467686023976/c6e6b0aaff0740d88bb308d228875d5e/artifacts
Train period: 2011-01-02 00:00:00 - 201